<a href="https://colab.research.google.com/github/bipulsimkhada/Transer-Learning--MNIST/blob/main/Transfer_Learning_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Transfer Learning using MNIST data
To illustrate the power and concept of transfer learning, we will train a CNN on just the digits 5,6,7,8,9.  Then we will train just the last layer(s) of the network on the digits 0,1,2,3,4 and see how well the features learned on 5-9 help with classifying 0-4.



In [ ]:


import datetime
import keras
import numpy as np
import tensorflow as tf
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [ ]:
#used to help some of the timing functions
now = datetime.datetime.now

In [ ]:
# set some parameters
batch_size = 128
num_classes = 5
epochs = 5

In [ ]:
# set some more parameters
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

In [ ]:
input_shape = (img_rows, img_cols,1)

In [ ]:
## To simplify things, write a function to include all the training steps
## As input, function takes a model, training set, test set, and the number of classes
## Inside the model object will be the state about which layers we are freezing and which we are training

def train_model(model, train, test, num_classes):
  #Reshape the data
  x_train = train[0].reshape((train[0].shape[0],)+ input_shape)
  x_test = test[0].reshape((test[0].shape[0],)+ input_shape)

  #Normalize the data
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')
  x_train = x_train/255
  x_test = x_test/255

  print('x_train shape: ', x_train.shape)
  print(x_train.shape[0],'train samples')
  print(x_test.shape[0], 'test samples')

  # One hot encode the targert label
  y_train = keras.utils.to_categorical(train[1], num_classes)
  y_test = keras.utils.to_categorical(test[1], num_classes)

  # Compile the model
  model.compile(loss='categorical_crossentropy', optimizer = 'adadelta', metrics = ['accuracy'])

  # Train the model on the training data
  t = now()
  model.fit(x_train, y_train, batch_size = batch_size, epochs = epochs,
            verbose =1, validation_data = (x_test,y_test))
  print('Training time: %s'% (now()-t))

  # Evaluate the model on the testing data
  score = model.evaluate(x_test,y_test, verbose = 0)
  print('Test score: ', score[0])
  print('Test accuracy: ', score[1])



In [ ]:
# Load the Mnist data and split between train and test sets
(x_train,y_train),(x_test,y_test) = mnist.load_data()
# create two datasets: one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train<5]
x_train_gte5 = x_train[y_train>=5]

y_train_lt5 = y_train[y_train<5]
y_train_gte5 = y_train[y_train>=5]-5

x_test_lt5 = x_test[y_test<5]
x_test_gte5 = x_test[y_test>=5]

y_test_lt5 = y_test[y_test<5]
y_test_gte5 = y_test[y_test>=5]-5

In [ ]:
# Define the "feature" layers. Add 2 convolution layer with max pool layer. At the end, add dropout layer with 0.25% probability and end with the flatten layer. These are the early layers that we expect will "transfer"
# to a new problem.  We will freeze these layers during the fine-tuning process

feature_layers = [
  Conv2D(filters, kernel_size,
         padding = 'valid',
         input_shape = input_shape),
  Activation('relu'),
  Conv2D(filters, kernel_size),
  Activation('relu'),
  MaxPooling2D(pool_size=pool_size),
  Dropout(0.25),
  Flatten()
]

In [ ]:
# Define the "classification" layers. Add Dense layer with 128 nodes and the output dense layer. These are the later layers that predict the specific classes from the features
# learned by the feature layers.  This is the part of the model that needs to be re-trained for a new problem

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')

]

In [ ]:
# Create the model by combining the two sets of layers:
model = Sequential(feature_layers + classification_layers)

In [ ]:
# print model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 activation_12 (Activation)  (None, 26, 26, 32)        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_13 (Activation)  (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 12, 12, 32)        0         
                                                      

In [ ]:
# Now, let's train our model on the digits 5,6,7,8,9
train_model (model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)


x_train shape:  (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 35s 150ms/step - loss: 1.5998 - accuracy: 0.2665 - val_loss: 1.5742 - val_accuracy: 0.3699
Epoch 2/5
230/230 [==============================] - 35s 150ms/step - loss: 1.5669 - accuracy: 0.3121 - val_loss: 1.5354 - val_accuracy: 0.3845
Epoch 3/5
230/230 [==============================] - 35s 150ms/step - loss: 1.5298 - accuracy: 0.3670 - val_loss: 1.4931 - val_accuracy: 0.4546
Epoch 4/5
230/230 [==============================] - 35s 151ms/step - loss: 1.4921 - accuracy: 0.4234 - val_loss: 1.4472 - val_accuracy: 0.5775
Epoch 5/5
230/230 [==============================] - 35s 154ms/step - loss: 1.4504 - accuracy: 0.4903 - val_loss: 1.3974 - val_accuracy: 0.6692
Training time: 0:03:22.642624
Test score:  1.3973960876464844
Test accuracy:  0.6692038774490356


### Freezing Layers
Keras allows layers to be "frozen" during the training process.  That is, some layers would have their weights updated during the training process, while others would not.  This is a core part of transfer learning, the ability to train just the last one or several layers.

Note also, that a lot of the training time is spent "back-propagating" the gradients back to the first layer.  Therefore, if we only need to compute the gradients back a small number of layers, the training time is much quicker per iteration.  This is in addition to the savings gained by being able to train on a smaller data set.

In [ ]:
# Freeze only the feature layers
for l in feature_layers:
  l.trainable = False

Observe below the differences between the number of *total params*, *trainable params*, and *non-trainable params*.

In [ ]:
# print model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 activation_12 (Activation)  (None, 26, 26, 32)        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_13 (Activation)  (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 12, 12, 32)        0         
                                                      

In [ ]:
# Now, let's train our model on the digits 0,1,2,3,4
train_model (model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape:  (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 13s 51ms/step - loss: 1.5776 - accuracy: 0.2888 - val_loss: 1.5322 - val_accuracy: 0.4374
Epoch 2/5
240/240 [==============================] - 12s 50ms/step - loss: 1.5227 - accuracy: 0.3659 - val_loss: 1.4756 - val_accuracy: 0.5336
Epoch 3/5
240/240 [==============================] - 12s 50ms/step - loss: 1.4721 - accuracy: 0.4299 - val_loss: 1.4211 - val_accuracy: 0.5978
Epoch 4/5
240/240 [==============================] - 12s 50ms/step - loss: 1.4217 - accuracy: 0.4886 - val_loss: 1.3672 - val_accuracy: 0.6385
Epoch 5/5
240/240 [==============================] - 12s 50ms/step - loss: 1.3728 - accuracy: 0.5422 - val_loss: 1.3150 - val_accuracy: 0.6602
Training time: 0:01:00.764486
Test score:  1.3150269985198975
Test accuracy:  0.6602451801300049


Note that after a single epoch, we are already achieving results on classifying 0-4 that are comparable to those achieved on 5-9 after 5 full epochs.  This despite the fact the we are only "fine-tuning" the last layer of the network, and all the early layers have never seen what the digits 0-4 look like.

Also, note that even though nearly all (590K/600K) of the *parameters* were trainable, the training time per epoch was still much reduced.  This is because the unfrozen part of the network was very shallow, making backpropagation faster. 